In [1]:
from tqdm import tqdm
import pandas as pd
import os
import gzip
import zipfile
import shutil
from time import sleep
import pymorphy2
from pyaspeller import YandexSpeller
from lru import LRU
import pickle
from time import sleep
from concurrent.futures import ThreadPoolExecutor
import tarfile
import bz2
import sys
import numpy as np
with open('ill_url_ids.pickle', 'rb') as file:
    ill_url_ids = pickle.load(file)
def transform(arg, ill_url_ids):
    if arg not in ill_url_ids:
        return arg
    else:
        return ill_url_ids[arg]
class DBN_info():
    def __init__(self):
        self.view = 0
        self.click = 0
        self.satis = 0

In [2]:
df_marks = pd.read_csv('train.marks.tsv', delimiter='\t', header=None)
df_marks = df_marks.rename(columns={0: "QueryId", 1: "DocumentId"})
df_marks = df_marks.drop(columns=[2])
df_example = pd.read_csv('sample.csv')
all_groups = df_marks.append(df_example)
all_groups = all_groups.reset_index()
all_groups = all_groups.drop(columns=['index'])
all_groups = all_groups.sort_values(by=['QueryId', 'DocumentId']).reset_index()
all_groups = all_groups.drop(columns=['index'])
rev_frame = all_groups.sort_values(by=['DocumentId', 'QueryId']).reset_index()
rev_frame = rev_frame.drop(columns=['index'])

In [3]:
q_ids = all_groups['QueryId'].unique()

In [4]:
q_docs_act = dict()

In [5]:
for qid in tqdm(q_ids):
    q_docs_act[qid] = set()
    tmp = all_groups[all_groups.QueryId==qid]
    tmp_docs = tmp['DocumentId'].unique()
    for doc in tmp_docs:
        q_docs_act[qid].add(transform(doc, ill_url_ids))

100%|██████████| 6311/6311 [00:09<00:00, 638.96it/s]


In [6]:
with open("q_docs_act.pickle", 'wb') as file:
    pickle.dump(q_docs_act, file)

In [7]:
double_q_dict = dict()

In [8]:
for key in tqdm(q_docs_act):
    double_q_dict[key] = dict()
    for elem in q_docs_act[key]:
        double_q_dict[key][elem]=DBN_info()

100%|██████████| 6311/6311 [00:01<00:00, 4780.95it/s]


In [9]:
exists = set(os.listdir('url_query'))
for key1 in tqdm(double_q_dict):
    if 'q_{}.pickle'.format(key1) in exists:
        with open('url_query/q_{}.pickle'.format(key1), 'rb') as file:
            obj = pickle.load(file)
            for session in obj:
                clicked_set = set()
                for num in session[3]:
                    clicked_set.add(num)
                if len(clicked_set)>0:
                    last_clicked = session[3][-1]
                    for num in session[2]:
                        if num not in clicked_set and num in q_docs_act[key1]:
                            double_q_dict[key1][num].view+=1
                        elif num in clicked_set:
                            if num in q_docs_act[key1] and num!=last_clicked:
                                double_q_dict[key1][num].view+=1
                                double_q_dict[key1][num].click+=1
                            elif num in q_docs_act[key1] and num==last_clicked:
                                double_q_dict[key1][num].view+=1
                                double_q_dict[key1][num].click+=1
                                double_q_dict[key1][num].satis+=1
                            if num==last_clicked:
                                break

100%|██████████| 6311/6311 [00:55<00:00, 113.44it/s]


In [15]:
with open("QDP_dbn.pickle", 'wb') as file:
    pickle.dump(double_q_dict, file)